# DriHRO prototype

This is a HRO prototype.

It should do:
    
    1. Get Ganttic projects
    2. Get CRM projects
    3. Reconcile both and launch warnings if anything is amiss.
    4. (Optional) Modify and edit stuff automatically
    5. (Optional) Get Factorial availability
    6. Generate a viz of available resources
    7. Generate a forecast of future income
    

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import datetime

import driganttic.client as dg_client
import driganttic.parse as dg_parse

In [2]:
def to_df(ResourceList):
    """Parses resourcelist to dataframe"""
    return pd.DataFrame(data=ResourceList.dict()['fetched_items'])
def filter_tasks(tasks, field='end', date= datetime.datetime.today()):
    m = tasks.end > date
    return tasks.loc[m]

# Get Ganttic data

In [3]:
# API KEY is stored in the env file
load_dotenv()
APIKEY = os.getenv("APIKEY")

In [4]:
Client = dg_client.GantticClient(APIKEY=APIKEY)
# get all projects, tasks and resources
projects = Client.get_projects()
tasks = Client.get_tasks(timeMin = dg_parse.parse_timestamp('2021-01-01'),
                         timeMax= dg_parse.parse_timestamp('2022-04-30'))
resources = Client.get_resources()

ValidationError: 1 validation error for ProjectDetails
scenario
  value is not a valid enumeration member; permitted: 'Esperat', 'Optimista', 'Congelat' (type=type_error.enum; enum_values=[<ScenarioEnum.esperat: 'Esperat'>, <ScenarioEnum.optimista: 'Optimista'>, <ScenarioEnum.congelat: 'Congelat'>])

In [ ]:
df_pro = to_df(projects)
df_res = to_df(resources)
df_tas = to_df(tasks)

df_pro.set_index('id',drop=True, inplace=True)
df_tas.set_index('id',drop=True, inplace=True)
df_res.set_index('id',drop=True, inplace=True)

In [ ]:
df_pro.head(1)

In [ ]:
df_res.head(1)

In [ ]:
df_pro.head(1)

In [ ]:
# Only active tasks today
df_tas = filter_tasks(df_tas)
df_tas['resources'] = df_tas.resources.map(lambda x: [df_res.loc[k]['name'] for k in x])
df_tas['project'] = df_tas.projectId.map(lambda x: df_pro.loc[x]['name'])
df_tas.head(1)

In [ ]:
# perform checks
# No tasks with more than one person
assert((df_tas.resources.map(len) > 1).sum() == 0)
# No project without task
assert((df_tas.project.isna().sum() == 0))
# Total people
print('People role distribution')
print(df_res.rol.value_counts())
print('\n')
# Tasks per project
print('Tasks per project')
print(df_tas.project.value_counts())
# Types of project
print('\n')
print('Types of project')
print(df_pro.service.value_counts())

In [ ]:
df_pro